In [1]:
import pandas as pd
import requests

In [2]:
# init watcher. 
# WARNING! Actualise api_key every 24h
api_key = 'RGAPI-a7d8b01e-b9fb-4024-b27c-c4f84c9531c5'

In [3]:
regions = [
    'br1', 
    'eun1', 
    'euw1', 
    'jp1', 
    'kr', 
    'la1', 
    'la2', 
    'na1', 
    'oc1', 
    'ph2', 
    'ru', 
    'sg2', 
    'th2', 
    'tr1', 
    'tw2', 
    'vn2'
]

In [4]:
df = pd.DataFrame(columns=[
                            'puuid', 'region', 'tier', 'placement', 'pair_id',
                            'champion_name', 'champion_rarity', 'champion_tier', 
                            'item_1', 'item_2', 'item_3', 
                            'augment_1', 'augment_2', 'augment_3'
                        ])

In [5]:
def accumulate_region_data(api_key, region, tier):
    players = []
    result = requests.get(f'https://{region}.api.riotgames.com/tft/league/v1/{tier}?queue=RANKED_TFT&page=1', headers={"X-Riot-Token": api_key})
    while result.status_code != 200:
        result = requests.get(f'https://{region}.api.riotgames.com/tft/league/v1/{tier}?queue=RANKED_TFT&page=1', headers={"X-Riot-Token": api_key})
        
    for player in result.json()['entries']:
        player_info = {
            'summonerId': player['summonerId'],
            'region': region,  # Sprawdzamy, czy region jest poprawnie przypisany
            'tier': tier
        }
        players.append(player_info)
    return players

In [6]:
def add_puuid(api_key, players, region):
    for index, player in enumerate(players):
        result = requests.get(f'https://{player["region"]}.api.riotgames.com/tft/league/v1/entries/by-summoner/{player["summonerId"]}', headers={"X-Riot-Token": api_key})
        while result.status_code != 200:
            result = requests.get(f'https://{player["region"]}.api.riotgames.com/tft/league/v1/entries/by-summoner/{player["summonerId"]}', headers={"X-Riot-Token": api_key})
        players[index]['puuid'] = result.json()[0]['puuid']
    return players

In [7]:
import os

In [17]:
def save_matches(api_key, players, bigger_region, region, tier):
    for player in players:
        result = requests.get(f'https://{bigger_region}.api.riotgames.com/tft/match/v1/matches/by-puuid/{player["puuid"]}/ids?start=0&count=100', headers={"X-Riot-Token": api_key})
        print('can_get?', result.status_code)
        while result.status_code != 200: 
            result = requests.get(f'https://{bigger_region}.api.riotgames.com/tft/match/v1/matches/by-puuid/{player["puuid"]}/ids?start=0&count=100', headers={"X-Riot-Token": api_key})
        print('getted')
        
        for match in result.json():
            if f'{match}.csv' in os.listdir('07.06.2024'):
                continue

            print('can_get result?', result.status_code)
            result = requests.get(f'https://{bigger_region}.api.riotgames.com/tft/match/v1/matches/{match}', headers={"X-Riot-Token": api_key})
            while result.status_code != 200:
                result = requests.get(f'https://{bigger_region}.api.riotgames.com/tft/match/v1/matches/{match}', headers={"X-Riot-Token": api_key})
            print('can_get result done')
            print(result.json()['info']['game_version'])
            if result.json()['info']['tft_game_type'] != 'standard' or result.json()['info']['tft_set_core_name'] !=  'TFTSet11' or result.json()['info']['game_version'] != "Linux Version 14.11.589.9418 (May 30 2024/09:53:23) [PUBLIC] <Releases/14.11>":
                continue

            df = pd.DataFrame(columns=[
                            'puuid', 'bigger_region', 'region', 'tier', 'placement',
                            'champion_name', 'champion_rarity', 'champion_tier', 
                            'item_1', 'item_2', 'item_3', 
                            'augment_1', 'augment_2', 'augment_3'
                        ])

            
            for participant in result.json()['info']['participants']:
                while len(participant['augments']) < 3:
                    participant['augments'].append(pd.NA)
                for unit in participant['units']:
                    while len(unit['itemNames']) < 3:
                        unit['itemNames'].append(pd.NA)

                    df.loc[df.shape[0]] = [
                                            participant['puuid'], bigger_region, region, tier, participant['placement'], 
                                            unit['character_id'], unit['rarity'], unit['tier'],
                                            unit['itemNames'][0], unit['itemNames'][1], unit['itemNames'][2],
                                            participant['augments'][0], participant['augments'][1], participant['augments'][2]
                                          ]

            df.to_csv(f'07.06.2024/{match}.csv')
            print(f'{match}.csv is saved')
            print('=====================')

In [18]:
def get_tft_data(api_key, region, tier='challenger'):
    players = accumulate_region_data(api_key, region, tier)
    print(f'Accumulation of players in {tier} from {region} is done')
    players = add_puuid(api_key, players, region)
    print(f'Adding puuid to players is done.')
    regions_dict = {
                        'americas': ['br1', 'la1', 'la2', 'na1'],
                        'europe': ['eun1', 'euw1', 'ru'],
                        'asia': ['jp1', 'kr', 'ph2', 'sg2', 'th2', 'tr1', 'tw2', 'vn2'],
                        'sea': ['oc1']
                    }

    for key, value in zip(regions_dict.keys(), regions_dict.values()):
        if region in value:
            bigger_region = key
            print('Start:', key)
            save_matches(api_key, players, bigger_region, region, tier)

    print('Done')
    
    

In [ ]:
for tier in ['challenger', 'grandmaster', 'master']:
    for region in regions:
        get_tft_data(api_key, region, tier)

Accumulation of players in challenger from br1 is done
Adding puuid to players is done.
Start: americas
can_get? 200
getted
can_get result? 200
can_get result done
Linux Version 14.11.589.9418 (May 30 2024/09:53:23) [PUBLIC] <Releases/14.11>
BR1_2947770915.csv is saved
can_get result? 200
can_get result done
Linux Version 14.11.589.9418 (May 30 2024/09:53:23) [PUBLIC] <Releases/14.11>
BR1_2947752032.csv is saved
can_get result? 200
can_get result done
Linux Version 14.11.589.9418 (May 30 2024/09:53:23) [PUBLIC] <Releases/14.11>
BR1_2947711446.csv is saved
can_get result? 200
can_get result done
Linux Version 14.11.589.9418 (May 30 2024/09:53:23) [PUBLIC] <Releases/14.11>
BR1_2947693183.csv is saved
can_get result? 200
can_get result done
Linux Version 14.11.589.9418 (May 30 2024/09:53:23) [PUBLIC] <Releases/14.11>
BR1_2947655791.csv is saved
can_get result? 200
can_get result done
Linux Version 14.11.589.9418 (May 30 2024/09:53:23) [PUBLIC] <Releases/14.11>
BR1_2947639256.csv is saved
